# <font color='FF6F1F'>Case Gol</font>

## <font color='FF6F1F'>Respostas às perguntas de interesse</font>

# Tabela de conteúdo

[1. Informação básica](#1.-Informação-básica)

[2. Bibliotecas](#2.-Bibliotecas)

[3. Dados](#3.-Dados)

[4. Perguntas](#4.-Perguntas)

* [1. Faça um ranking para o número total de PAX por dia da semana.](#1.-Faça-um-ranking-para-o-número-total-de-PAX-por-dia-da-semana.)

* [2. Qual a correlação de sábado e domingo somados com o total de RPK?](#2.-Qual-a-correlação-de-sábado-e-domingo-somados-com-o-total-de-RPK?)

* [3. Qual a média de ‘Monetário’ por mês por Canal? E a mediana?](#3.-Qual-a-média-de-‘Monetário’-por-mês-por-Canal?-E-a-mediana?)

* [4. Crie um forecast de PAX por ‘Local de Venda’ para os próximos 15 dias a contar da última data de venda.](#4.-Crie-um-forecast-de-PAX-por-‘Local-de-Venda’-para-os-próximos-15-dias-a-contar-da-última-data-de-venda.)

* [5. Supondo que você precisa gerar um estudo para a área responsável, com base em qualquer modelo ou premissa, qual ‘Local de Venda’ você considera mais crítico. Por quê?](#5.-Supondo-que-você-precisa-gerar-um-estudo-para-a-área-responsável,-com-base-em-qualquer-modelo-ou-premissa,-qual-‘Local-de-Venda’-você-considera-mais-crítico.-Por-quê?)

* [6. Criar modelo relacionando o comportamento de venda com variáveis não apresentadas nos dados (Ex: PIB, Dolar, etc.)](#6.-Criar-modelo-relacionando-o-comportamento-de-venda-com-variáveis-não-apresentadas-nos-dados-(Ex:-PIB,-Dolar,-etc.))


## 1. Informação básica
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

**Fonte:** Gol

**Período:** 2016-11-01 a 2017-04-01

**Descrição das colunas:**

| Variável | Descrição | Tipo |  
|:---|:---|:---|
| Data Venda | Data (AAAA-MM-DD) | Data |
| Canal de Venda | Canais de venda. | Nominal |
| Local de Venda | Locais de venda. | Nominal |
| PAX | Total de passageiros. | Discreto |
| Monetário Vendido | Valores da venda. | Contínuo |
| RPK | Indicador diretamente relacionado com o número de PAX. | Discreto |

## 2. Bibliotecas
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [1]:
import pandas as pd # dataframe
import numpy as np # numeric
import seaborn as sns # graph
from matplotlib import pyplot as plt # plot

## 3. Dados
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [2]:
df = pd.read_excel('case_analytics-ds.xlsx', sheet_name='Dados')

In [13]:
df.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK
199,2016-11-13,TeleVenda,Vast,299,20202.737369,14597
1058,2017-01-06,Telégrafo,Arena,128,7622.828531,18140
2113,2017-03-13,TeleVenda,Ellipsis,121,6667.114100,7561
570,2016-12-06,Telégrafo,Arena,86,8814.144563,12883
2027,2017-03-07,TeleVenda,Arena,1224,36059.215517,180104


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2432 entries, 0 to 2431
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Data Venda         2432 non-null   datetime64[ns]
 1   Canal de Venda     2432 non-null   object        
 2   Local de Venda     2432 non-null   object        
 3   PAX                2432 non-null   int64         
 4   Monetário Vendido  2432 non-null   float64       
 5   RPK                2432 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 114.1+ KB


In [18]:
print(f'O dataframe possui {df.shape[0]} linhas e {df.shape[1]} colunas.')

O dataframe possui 2432 linhas e 6 colunas.


In [8]:
# Dados faltantes
total = df.isnull().sum().sort_values(ascending=False)
porcentagem = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
faltante = pd.concat([total, porcentagem], axis=1, keys=['Total', 'Porcentagem'])
faltante.head(45)

,Total,Porcentagem
RPK,0,0.0
Monetário Vendido,0,0.0
PAX,0,0.0
Local de Venda,0,0.0
Canal de Venda,0,0.0
Data Venda,0,0.0


* Não há dados faltantes.

In [11]:
# Lista dos canais de venda
df['Canal de Venda'].unique().tolist()

['Telégrafo', 'TeleVenda', 'Porta a Porta']

In [12]:
# Lista dos locais de venda
df['Local de Venda'].unique().tolist()

['Mindscape', 'Arena', 'Vast', 'Ellipsis']

In [17]:
# Período dos dados
print(f"Início do período em análise: {df['Data Venda'].min()}.")
print(f"Fim do período em análise: {df['Data Venda'].max()}.")

Início do período em análise: 2016-11-01 00:00:00.
Fim do período em análise: 2017-04-01 00:00:00.


## 4. Perguntas
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

### 1. Faça um ranking para o número total de PAX por dia da semana.
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [114]:
# Cópia do dataframe
df_ = df.copy()

In [115]:
# Cria dia da semana
df_['dia_semana'] = df_['Data Venda'].dt.day_name()

In [116]:
# en --> pt
df_.loc[df_['dia_semana'] == 'Monday', 'dia_semana'] = 'segunda-feira'
df_.loc[df_['dia_semana'] == 'Tuesday', 'dia_semana'] = 'terça-feira'
df_.loc[df_['dia_semana'] == 'Wednesday', 'dia_semana'] = 'quarta-feira'
df_.loc[df_['dia_semana'] == 'Thursday', 'dia_semana'] = 'quinta-feira'
df_.loc[df_['dia_semana'] == 'Friday', 'dia_semana'] = 'sexta-feira'
df_.loc[df_['dia_semana'] == 'Saturday', 'dia_semana'] = 'sábado'
df_.loc[df_['dia_semana'] == 'Sunday', 'dia_semana'] = 'domingo'

In [117]:
df_.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana
16,2016-11-02,TeleVenda,Ellipsis,1007,29177.233024,54696,quarta-feira
2431,2017-04-01,Telégrafo,Vast,61,2911.981488,3050,sábado
1083,2017-01-07,TeleVenda,Arena,1988,73393.342549,294170,sábado
1766,2017-02-19,Porta a Porta,Mindscape,1774,43890.144663,208521,domingo
1728,2017-02-17,Porta a Porta,Vast,1052,27193.584274,50576,sexta-feira


In [118]:
x = df_.groupby(by='dia_semana')['PAX'].sum().sort_values(ascending=False)
x

dia_semana
quarta-feira     293025
quinta-feira     258488
sexta-feira      255625
sábado           253467
domingo          241620
segunda-feira    236316
terça-feira      219802
Name: PAX, dtype: int64

* O dia com maior número de passageiros, em todo período, é quarta-feira.

In [119]:
pax_dia = df_.groupby('dia_semana').agg({'PAX': ['sum']})
pax_dia

,PAX
,sum
dia_semana,
domingo,241620
quarta-feira,293025
quinta-feira,258488
segunda-feira,236316
sexta-feira,255625
sábado,253467
terça-feira,219802


### 2. Qual a correlação de sábado e domingo somados com o total de RPK?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [121]:
# Dia da semana: segunda-feira == 0; domingo == 6
df_['dia_semana_num'] = df_['Data Venda'].dt.dayofweek

In [122]:
df_.sample(3)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana,dia_semana_num
2004,2017-03-06,Porta a Porta,Arena,1133,38691.351623,157987,segunda-feira,0
1315,2017-01-22,Porta a Porta,Ellipsis,636,20063.679977,35053,domingo,6
1897,2017-02-27,TeleVenda,Arena,1355,38592.633873,198075,segunda-feira,0


In [124]:
df_[['RPK', 'dia_semana_num']].corr()

,RPK,dia_semana_num
RPK,1.000000,0.000134
dia_semana_num,0.000134,1.000000


In [141]:
q = df_.groupby('dia_semana_num').agg({'RPK': ['sum']})
q

,RPK
,sum
dia_semana_num,
0,23275908
1,20366840
2,26263705
3,23501102
4,23309936
5,23201577
6,22388695


### 3. Qual a média de ‘Monetário’ por mês por Canal? E a mediana?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [146]:
# Coluna 'mês'
df_['mês'] = df['Data Venda'].dt.month

In [147]:
df_.sample(5)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK,dia_semana,dia_semana_num,mês
1828,2017-02-23,Telégrafo,Ellipsis,61,2372.902848,3427,quinta-feira,3,2
81,2016-11-06,Porta a Porta,Arena,1078,50722.959127,147535,domingo,6,11
150,2016-11-10,TeleVenda,Mindscape,124,8177.826736,15080,quinta-feira,3,11
857,2016-12-24,TeleVenda,Arena,186,14147.785011,29776,sábado,5,12
58,2016-11-04,TeleVenda,Mindscape,112,8420.150405,13803,sexta-feira,4,11


In [149]:
# Monerário Vendido por Canal de Venda
df_.groupby('Canal de Venda').agg({'Monetário Vendido': ['mean', 'median']})

Monetário Vendido              
                            mean        median
Canal de Venda                                
Porta a Porta       31454.583113  29281.731170
TeleVenda           31256.502520  18574.866487
Telégrafo            4587.373839   4105.413682

In [150]:
df_.groupby([['Canal de Venda', 'mês']]).agg({'Monetário Vendido': ['mean', 'median']})

ValueError: Grouper and axis must be same length

### 4. Crie um forecast de PAX por ‘Local de Venda’ para os próximos 15 dias a contar da última data de venda.
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

### 5. Supondo que você precisa gerar um estudo para a área responsável, com base em qualquer modelo ou premissa, qual ‘Local de Venda’ você considera mais crítico. Por quê?
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

### 6. Criar modelo relacionando o comportamento de venda com variáveis não apresentadas nos dados (Ex: PIB, Dolar, etc.)
[Retorna à Tabela de conteúdo](#Tabela-de-conteúdo)

In [ ]:
Utilizando como base de dados na aba dados , responda as seguintes perguntas:

1-Faça um ranking para o número total de PAX por dia da semana.


2-Qual a correlação de sábado e domingo somados com o total de RPK?

3-Qual a média de ‘Monetário’ por mês por Canal? E a mediana?

4-Crie um forecast de PAX por ‘Local de Venda’ para os próximos 15 dias a contar da última data de venda. (Aqui a técnica é livre) 

5-Supondo que você precisa gerar um estudo para a área responsável, com base em qualquer modelo ou premissa, qual ‘Local de Venda’ você considera mais crítico. Por quê?

6-Criar modelo relacionando o comporatamento de venda com variaveis não apresentada nos dados (Ex : PIB, Dolar, e etc)


In [ ]:
Mais do que o resultado, iremos avaliar o raciocínio e a organização de trabalho

Algumas considerações:

*PAX é o total de passageiros. RPK é um indicador diretamente relacionada com o número de PAX.
*Não se atenha às grandezas. Os dados são fictícios. 😉
*Envie todo o material que produzir (códigos, tabelas e outros arquivos) com o detalhamento de cada um. Se possível, comente nos códigos.
*Para a apresentação, use PowerPoint ou qualquer outra ferramenta de DataViz que julgar pertinente.
